In [1]:
import os
from typing import List

import chess
import pandas as pd
import numpy as np
from reconchess import Turn
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Conv2D, Activation, Dropout, LSTM, Dense, Reshape, Masking
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import BatchNormalization

import tensorflow as tf

from tqdm import tqdm

from game_log_reader.game_history_extended import GameHistoryExtended

C:\Users\User\AppData\Roaming\Python\Python310\site-packages\pydot.py:17: UserWarning: `pydot` could not import `dot_parser`, so `pydot` will be unable to parse DOT files. The error was:  No module named 'pyparsing'
  warnings.warn(


In [2]:
history = GameHistoryExtended.from_file('game_logs/historical_games_extended/571698.json')
color = chess.WHITE
move = 1

In [3]:
def int_to_square(squares):
    board = np.zeros(64)
    if squares is not None:
        board[squares] = 1
    return board.reshape((8, 8))

In [4]:
def parse_position(board, color):
    return [np.reshape([board.pieces_mask(piece_type, color) >> i & 1 for i in range(64)], (8, 8)) for piece_type in
            chess.PIECE_TYPES]

In [5]:
def map_move(history: GameHistoryExtended, turn: Turn):
    color = turn.color
    move = turn.turn_number

    # 1 Color Layer
    color_layer = np.ones((1, 8, 8)) if color else np.zeros((1, 8, 8))
    # 1 Sense Location Layer
    sense_location = np.array([int_to_square(history.sense(turn))])
    # 6 Sense Result Layers - one for each piece type
    sense_res = np.array([
        int_to_square([
            result[0] for result in history.sense_result(turn) if
            (result[1] is not None and result[1].piece_type == piece_type)
        ]) for piece_type in chess.PIECE_TYPES
    ])

    # 1 Layer for piece captured on previous turn
    opponent_capture_square = np.zeros((1, 8, 8)) if turn == Turn(chess.WHITE, 0) else \
        np.array([int_to_square(history.capture_square(turn.previous))])

    # 6 Layers for current state of owned pieces
    owned_piece_positions = np.array([
        np.reshape([history.truth_board_before_move(turn).pieces_mask(piece_type, color) >> i & 1 for i in range(64)],
                   (8, 8))
        for piece_type in chess.PIECE_TYPES
    ])

    # 6 Layers for last move origin - one for each piece type
    last_move_origin = np.zeros((6, 8, 8))

    # 1 Layer for last move requested destination
    last_move_requested_destination = np.zeros((1, 8, 8))

    # 3 Layers for requested under-promotions
    under_promotions = np.zeros((3, 8, 8))

    # 1 Layer for last move taken destination
    last_move_taken_destination = np.zeros((1, 8, 8))

    # 1 Layer for captured piece
    captured_square = np.zeros((1, 8, 8))

    if move > 0:
        prev_turn = turn.previous.previous
        prev_requested_move = history.requested_move(prev_turn)
        if prev_requested_move is not None:
            prev_piece_type = history.truth_board_before_move(prev_turn).piece_at(prev_requested_move.from_square).piece_type
            last_move_origin[chess.PIECE_TYPES.index(prev_piece_type)] = int_to_square(prev_requested_move.from_square)
            last_move_requested_destination = np.array([int_to_square(prev_requested_move.to_square)])

            if prev_requested_move.promotion not in [None, chess.QUEEN]:
                under_promotions[[chess.ROOK, chess.KNIGHT, chess.BISHOP].index(prev_requested_move.promotion)] = np.ones((8, 8))

        if history.capture_square(prev_turn) is not None:
            captured_square = [int_to_square(history.capture_square(prev_turn))]
        taken_move = history.taken_move(prev_turn)
        if taken_move is not None:
            last_move_taken_destination = np.array([int_to_square(taken_move.to_square)])

    next_requested_move = history.requested_move(turn)
    if next_requested_move is not None:
        next_piece_type = history.truth_board_before_move(turn).piece_at(next_requested_move.from_square).piece_type

        # 6 Layers for next move origin - one for each piece type
        next_move_origin = np.array([
            int_to_square(next_requested_move.from_square if next_piece_type == piece_type else None)
            for piece_type in chess.PIECE_TYPES
        ])

        # 1 Layer for last move requested destination
        next_move_requested_destination = np.array([int_to_square(next_requested_move.to_square)])

        # 3 Layers for attempted underpromotions
        next_under_promotions = np.zeros((3, 8, 8))
        if next_requested_move.promotion not in [None, chess.QUEEN]:
            under_promotions[[chess.ROOK, chess.KNIGHT, chess.BISHOP].index(next_requested_move.promotion)] = np.ones((8, 8))
    else:
        next_move_origin = np.zeros((6, 8, 8))
        next_move_requested_destination = np.zeros((1, 8, 8))
        next_under_promotions = np.zeros((3, 8, 8))

    return np.concatenate(
        (color_layer,  # 0
         sense_location,  # 1
         sense_res, # 2-7
         opponent_capture_square, # 8
         last_move_origin, # 9-14
         last_move_requested_destination, # 15
         last_move_taken_destination, # 16
         captured_square, # 17
         under_promotions, # 18-20
         owned_piece_positions, # 21-26
         next_move_origin, # 27-32
         next_move_requested_destination, #33
         next_under_promotions, #34-36
         )
    )

In [6]:
def map_game(history: GameHistoryExtended, color: chess.Color):
    if history._post_sense_uncertainty[not color] is None:
        return None

    measured_turns = len(history._post_sense_uncertainty[not color]) - int(not color)

    if measured_turns<=0:
        return None

    turns = [Turn(color, move) for move in range(measured_turns)]

    X = np.array([map_move(history, turn) for turn in turns])
    X = np.transpose(X, (0, 2, 3, 1))

    y = np.array(history._post_sense_uncertainty[not color][int(not color):])
    y = np.log(y)/12.0

    return X, y

In [7]:
game_map = map_game(history, chess.WHITE)
game_map

(array([[[[1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.]],
 
         [[1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.]],
 
         [[1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          [1., 0., 0., ..., 0., 0., 0.],
          ...,
          [1., 0., 0., ..., 0., 0., 0.],
          [1.

In [8]:
input_dim = (8, 8, 37)

In [9]:
def cnn_lstm(input_dim, dropout=0.1):

    # Input data type
    dtype = 'float32'

    # ---- Network model ----
    input_data = Input(name='input', shape=input_dim, dtype=dtype)

    x = Masking(mask_value=-1)(input_data)

    x = Conv2D(filters=128, kernel_size=3, name='conv_1')(x)
    x = BatchNormalization(name='norm_1')(x)
    x = Activation('relu', name='activation_1')(x)
    x = Dropout(dropout, name='dropout_1')(x)

    x = Conv2D(filters=256, kernel_size=3, name='conv_2')(x)
    x = BatchNormalization(name='norm_2')(x)
    x = Activation('relu', name='activation_2')(x)
    x = Dropout(dropout, name='dropout_2')(x)

    x = Reshape((-1, 4*4*256), name='reshape_1')(x)

    x = LSTM(128, activation='relu', return_sequences=True,
             dropout=dropout, name='lstm_1')(x)
    x = LSTM(128, activation='relu', return_sequences=True,
             dropout=dropout, name='lstm_2')(x)

    x = Dense(units=64, activation='relu', name='fc')(x)
    x = Dropout(dropout, name='dropout_3')(x)

    y_pred = Dense(1, name='output', activation='sigmoid')(x)

    network_model = Model(inputs=input_data, outputs=y_pred)

    return network_model

In [10]:
def masked_squared_error(y_true, y_pred):
    mask = tf.where(tf.not_equal(y_true, -1.0))
    return tf.reduce_mean(tf.square(tf.gather(y_true, mask) - tf.gather(y_pred, mask)))

In [11]:
model = cnn_lstm((None, *input_dim))

model.compile(loss=masked_squared_error, optimizer='adam')

The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm), but
are not present in its tracked objects:
  <tf.Variable 'norm_1/gamma:0' shape=(128,) dtype=float32>
  <tf.Variable 'norm_1/beta:0' shape=(128,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_1), but
are not present in its tracked objects:
  <tf.Variable 'norm_2/gamma:0' shape=(256,) dtype=float32>
  <tf.Variable 'norm_2/beta:0' shape=(256,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


In [12]:
model.load_weights('uncertainty_model/model2/weights')

In [46]:
from sklearn.model_selection import train_test_split

data_path = 'game_logs/historical_games_extended'

files = os.listdir(data_path)
files.sort()

train_data, test_data = train_test_split(files, test_size=0.2, random_state=42)

pd.DataFrame({'files': train_data}).to_csv('uncertainty_model/train_data.csv')
pd.DataFrame({'files': test_data}).to_csv('uncertainty_model/test_data.csv')

In [47]:
from tensorflow.python.keras.utils import data_utils

class GameHistorySequence(data_utils.Sequence):
    def __init__(self, files, data_path, batch_size=1, shuffle = True):
        self.data_path = data_path
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.files = files
        if self.shuffle:
            np.random.shuffle(self.files)

    def __len__(self):
        return int(np.ceil(len(self.files) / self.batch_size))

    def __getitem__(self, idx):
        files = self.files[idx*self.batch_size: min((idx+1)*self.batch_size, len(self.files))]

        histories = [GameHistoryExtended.from_file(os.path.join(self.data_path, file_name)) for file_name in files]

        data = [map_game(game, color) for game in histories for color in chess.COLORS]
        data = [e for e in data if e is not None]

        X, y = [e[0] for e in data], [e[1] for e in data]

        x_len = max(len(x) for x in X)
        X = np.array([np.concatenate((x, -1 * np.ones((x_len-x.shape[0], *x.shape[1:])))) for x in X])
        y = np.array([np.concatenate((e, -1 * np.ones((x_len-len(e))))) for e in y])

        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.files)


In [48]:
training_sequence = GameHistorySequence(train_data, data_path, 16)
test_sequence = GameHistorySequence(test_data, data_path, 16, shuffle=False)

In [49]:
from tensorflow.python.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('uncertainty_model/model2', 'val_loss', verbose=1, save_best_only=True, mode='max')

In [50]:
hist = model.fit(training_sequence, epochs=2, callbacks=checkpoint)

Epoch 1/2
282/282 [==============================] - 2842s 10s/step - loss: 0.4620
Epoch 2/2
282/282 [==============================] - 2785s 10s/step - loss: 0.4412


In [21]:
hist = model.fit(training_sequence, epochs=2, callbacks=checkpoint)

Epoch 1/2
282/282 [==============================] - 2794s 10s/step - loss: 0.6130
Epoch 2/2
282/282 [==============================] - 2771s 10s/step - loss: 0.6049


In [20]:
model.save('uncertainty_model/model1')

AssertionError: Tried to export a function which references an 'untracked' resource. TensorFlow objects (e.g. tf.Variable) captured by functions must be 'tracked' by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly. See the information below:
	Function name = b'__inference_signature_wrapper_6332'
	Captured Tensor = <ResourceHandle(name="Resource-2-at-0x1c61d010330", device="/job:localhost/replica:0/task:0/device:GPU:0", container="Anonymous", type="class tensorflow::Var", dtype and shapes : "[ DType enum: 1, Shape: [128] ]")>
	Trackable referencing this tensor = <tf.Variable 'norm_1/gamma:0' shape=(128,) dtype=float32>
	Internal Tensor = Tensor("6290:0", shape=(), dtype=resource)

In [51]:
model.save_weights('uncertainty_model/model2/weights')

In [52]:
import pickle
with open('uncertainty_model/model2/train_hist', 'wb') as file_pi:
    pickle.dump(hist.history, file_pi)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None, 8, 8, 37)]  0         
_________________________________________________________________
masking (Masking)            (None, None, 8, 8, 37)    0         
_________________________________________________________________
conv_1 (Conv2D)              (None, None, 6, 6, 128)   42752     
_________________________________________________________________
tf.cast (TFOpLambda)         (None, None, 6, 6, 128)   0         
_________________________________________________________________
tf.compat.v1.nn.fused_batch_ ((None, None, 6, 6, 128), 0         
_________________________________________________________________
activation_1 (Activation)    (None, None, 6, 6, 128)   0         
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 6, 6, 128)   0     

In [23]:
game_map[1]

array([0.21374578, 0.        , 0.        , 0.        , 0.21374578,
       0.        , 0.        , 0.        , 0.        , 0.25758687,
       0.23104906, 0.20707555, 0.24964436, 0.        , 0.22567085,
       0.13411983, 0.30740662, 0.23104906, 0.39320824, 0.11552453,
       0.1732868 , 0.43984289, 0.5259779 , 0.33085766, 0.37119561])

In [41]:
model.predict(np.array([game_map[0][0:1]]))

array([[[0.09206896]]], dtype=float32)

In [56]:
model.predict(test_sample)

array([[[9.65302661e-02],
        [1.84438210e-02],
        [1.08479137e-04],
        ...,
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[1.46613687e-01],
        [1.66606791e-02],
        [3.14770907e-04],
        ...,
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[9.01013464e-02],
        [1.72635075e-02],
        [8.06811586e-05],
        ...,
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00]],

       ...,

       [[1.16691105e-01],
        [1.57225449e-02],
        [8.28637785e-05],
        ...,
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[8.83309990e-02],
        [1.63214374e-02],
        [6.22796797e-05],
        ...,
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[1.09717838e-01],
        [1.31376991e-02],
        [2.62407644e-04],
        ...,
        [0.00000000e+00],
        [0.0000